In [1]:
# lets import basic python libraries
import numpy as np
import pandas as pd
import datetime

# for advanced data visualizations
import plotly
import plotly.graph_objects as go
import plotly.offline as py
from plotly.offline import init_notebook_mode, iplot, plot
import plotly.graph_objs as go
from plotly import tools
init_notebook_mode(connected = True)

# for jupyter notebook widgets
import ipywidgets as widgets
from ipywidgets import interact
from ipywidgets import interact_manual

In [2]:
# lets read the dataset
data = pd.read_csv('https://raw.githubusercontent.com/yankev/testing/master/datasets/nycflights.csv')
data = data.drop(data.columns[[0]], axis=1)

# lets check the shape of data
data.shape

(336776, 16)

In [3]:
# lets check the head of the dataset
data.head()

,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute
0,2013,1,1,517.0,2.0,830.0,11.0,UA,N14228,1545,EWR,IAH,227.0,1400,5.0,17.0
1,2013,1,1,533.0,4.0,850.0,20.0,UA,N24211,1714,LGA,IAH,227.0,1416,5.0,33.0
2,2013,1,1,542.0,2.0,923.0,33.0,AA,N619AA,1141,JFK,MIA,160.0,1089,5.0,42.0
3,2013,1,1,544.0,-1.0,1004.0,-18.0,B6,N804JB,725,JFK,BQN,183.0,1576,5.0,44.0
4,2013,1,1,554.0,-6.0,812.0,-25.0,DL,N668DN,461,LGA,ATL,116.0,762,5.0,54.0


In [4]:
@interact
def correlations(column1 = list(data.select_dtypes('number').columns),
                 column2 = list(data.select_dtypes('number').columns[1:])):
    print(f"Correlations: ", {data[column1].corr(data[column2])})


interactive(children=(Dropdown(description='column1', options=('year', 'month', 'day', 'dep_time', 'dep_delay'…

In [5]:

pd.set_option('max_rows', 30000)

@interact
def show_articles_more_than(column='dep_delay', x=60.0):
    return data.loc[data[column] > x]

interactive(children=(Text(value='dep_delay', description='column'), FloatSlider(value=60.0, description='x', …

In [12]:
month = widgets.IntSlider(
    value=1.0,
    min=1.0,
    max=12.0,
    step=1.0,
    description='Month:',
    continuous_update=False
)

use_date = widgets.Checkbox(
    description='Date: ',
    value=True,
)

container = widgets.HBox(children=[use_date, month])

textbox = widgets.Dropdown(
    description='Airline:   ',
    value='DL',
    options=data['carrier'].unique().tolist()
)

origin = widgets.Dropdown(
    options=list(data['origin'].unique()),
    value='LGA',
    description='Origin Airport:',
)


# Assign an empty figure widget with two traces
trace1 = go.Histogram(x=data['arr_delay'], opacity=0.75, name='Arrival Delays')
trace2 = go.Histogram(x=data['dep_delay'], opacity=0.75, name='Departure Delays')
g = go.FigureWidget(data=[trace1, trace2],
                    layout=go.Layout(
                        title=dict(
                            text='NYC FlightDatabase'
                        ),
                        barmode='overlay'
                    ))

In [13]:
def validate():
    if origin.value in data['origin'].unique() and textbox.value in data['carrier'].unique():
        return True
    else:
        return False


def response(change):
    if validate():
        if use_date.value:
            filter_list = [i and j and k for i, j, k in
                           zip(data['month'] == month.value, data['carrier'] == textbox.value,
                               data['origin'] == origin.value)]
            temp_df = data[filter_list]

        else:
            filter_list = [i and j for i, j in
                           zip(data['carrier'] == 'DL', data['origin'] == origin.value)]
            temp_df = data[filter_list]
        x1 = temp_df['arr_delay']
        x2 = temp_df['dep_delay']
        with g.batch_update():
            g.data[0].x = x1
            g.data[1].x = x2
            g.layout.barmode = 'overlay'
            g.layout.xaxis.title = 'Delay in Minutes'
            g.layout.yaxis.title = 'Number of Delays'


origin.observe(response, names="value")
textbox.observe(response, names="value")
month.observe(response, names="value")
use_date.observe(response, names="value")

In [14]:
container2 = widgets.HBox([origin, textbox])
widgets.VBox([container,
              container2,
              g])

In [18]:

@interact_manual
def scatter_plot(x = list(data.select_dtypes('number').columns),
                 y = list(data.select_dtypes('number').columns)[1:],
                ):
    data.plot(kind = 'scatter',
               x = x,
               y = y,
               )

interactive(children=(Dropdown(description='x', options=('year', 'month', 'day', 'dep_time', 'dep_delay', 'arr…